In [10]:
# 在运行其他代码前，请先运行此块代码！

%load_ext jupyter_spaces
%load_ext blackcellmagic

The jupyter_spaces extension is already loaded. To reload it, use:
  %reload_ext jupyter_spaces
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


[◀ 上一节](2.3.ipynb) [下一节 ▶](3.2.ipynb)

[回到目录](https://lechocolatchaud.github.io/ITClass/)

<br/>

*普通高中教科书 信息技术*

*必修一 数据与计算*

# 第三章 数据处理与应用 - 第一节

本节主要讲解数据采集、整理与安全。

## 数据采集

- 一般过程
    1. 明确数据要求
    2. 确定数据来源
    3. 选择采集方法
    4. 实时数据采集
- 数据来源
    - 传感设备
    - 互联网
    - 问卷调查
    - 企业内部数据库
    - ......
- 采集数据的方法
    - 传感数据采集
    - 互联网数据采集

### 互联网数据采集

#### 1. 获取网页

客户端：

1. 向服务器发送访问请求
2. 接收服务器响应的内容

**Python 实现**

Python 中 Requests 库能够让我们很方便地发送 HTTP 请求。

> [Requests 官方文档](https://requests.readthedocs.io/en/latest/api/)

> **小拓展**：HTTP 请求方法  
> GET：最常见的请求方法，一般用于获取或者查询资源信息  
> POST：以表单形式上传参数，可以查询信息或修改信息  
> PUT：上传或替换服务器端的某资源

In [11]:
%%space request_demo_1

import requests                                                    # 导入 Requests 库

url = "https://tool.520101.com/calculator/fushuchengfa/"           # 确定请求对象地址
html = requests.get(url)                                            # 用 GET 方式获取网页数据
html.encoding = "gbk"                                             # 用 GBK 文本编码
print(html.text)                                                    # 输出网页的源代码

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7" /> 
<meta http-equiv="Content-Type" content="text/html; charset=gb2312" />
<title>复数乘法在线计算器</title>
<meta name="keywords" content="复数,乘法,运算,虚数,推算" />
<meta name="description" content="一个在线复数乘法计算器工具，复数运算编程来执行两个复数乘法运算。复数a + bi形式，其中a和b都是实数的表达。如果z = a + bi的是一个复数，然后被称为a和b的实数和虚数的z，分别表示为（z）的和Im（z）的。使用复杂的数字在许多科学领域，包括工程，电磁学，量子物理学，应用数学理论 等。因此，有必要学习复杂的数字运算。复数乘法计算器示出的行为相对于基本相对应的乘法运算的实数和虚数的复数。" />
<link href="/css/skin.css" rel="stylesheet" type="text/css" />
<link href="/favicon.ico" rel="shortcut icon" />
</head>
<body>

<div class="topbg"><div class="w950 head1"><div class="flo_l logo"><a href="/" target="_self"><img src="/img/logo.gif" /></a></div>
 <div class="flo_r head2"><div class="rcor flo_r"><ul><li><img src="/img/icon1.gif" /><a href="https://tool.520101.com">

#### 2. 解析网页

通过 Requests 库抓取到网页源代码之后，我们要从源代码中找到并提取数据。

**Python 实现**

Beautiful Soup 库是 Python 的一个功能很强的库，其主要功能是从网页中抓取数据。

导入 Beautiful Soup 库的语法为:

In [12]:
%%space request_demo_2
from bs4 import BeautifulSoup                                      # 导入 Beautiful Soup 库

导入库后，可用 Beautiful Soup 库进行源代码解析，具体如下：

In [13]:
%%space request_demo_2

import requests                                                    # 导入 Requests 库

url = "https://tool.520101.com/calculator/fushuchengfa/"           # 确定请求对象地址
html = requests.get(url)                                            # 用 GET 方式获取网页数据
html.encoding = "gbk"                                             # 用 GBK 文本编码

soup = BeautifulSoup(html.text, 'html.parser')                      # 解析源代码

Beautiful Soup 库中常用以下方法获取特定网页内容：  
- `find`：返回第一个符合条件的标签

In [14]:
%%space request_demo_2
soup.find("a")

<a href="/" target="_self"><img src="/img/logo.gif"/></a>

- `find_all`：返回所有符合条件的标签

In [15]:
%%space request_demo_2
soup.find_all("a")

[<a href="/" target="_self"><img src="/img/logo.gif"/></a>,
 <a href="https://tool.520101.com">工具首页</a>,
 <a href="javascript:window.external.addfavorite('https://tool.520101.com','实用工具大全')">收藏本页</a>,
 <a href="/yuanqijian/">元器件类</a>,
 <a href="/dianlu/">电路计算</a>,
 <a href="/calculator/">科学计算器</a>,
 <a href="/rf/">RF/射频计算</a>,
 <a href="/changyong/">常用工具</a>,
 <a href="/wangluo/">网络工具</a>,
 <a class="xdhbg" href="/yuanqijian/5sehuandianzu/"><div class="xdh_img xico101"></div><div class="xdh_bt">5色环电阻阻值</div></a>,
 <a class="xdhbg" href="/yuanqijian/4sehuandianzu/"><div class="xdh_img xico102"></div><div class="xdh_bt">4色环电阻阻值</div></a>,
 <a class="xdhbg" href="/yuanqijian/sanjiguanpianzhi/"><div class="xdh_img xico94"></div><div class="xdh_bt">三极管偏置电压</div></a>,
 <a class="xdhbg" href="/yuanqijian/wenyaerjiguan/"><div class="xdh_img xico90"></div><div class="xdh_bt">稳压二极管应用</div></a>,
 <a class="xdhbg" href="/yuanqijian/jvzhidianrong/"><div class="xdh_img xico97"></div><div class="xdh_

- `select`：返回指定 CSS 样式的内容，或者输入 `text` 返回去除所有 HTML 标签后的内容

In [16]:
%%space request_demo_2
soup.select("div#challenge-body-text")
soup.select("title")

[<title>复数乘法在线计算器</title>]

> **小拓展**：HTML 和 CSS  
> HTML 是一门由标签组成的语言，其使用标签来描述网页。每一个标签都可以带有id和类名。  
> HTML 常见标签有：`a`, `p`, `div`, `title` 等。  
> CSS 是一门指定网页样式的语言。其通常用 `标签名.类名#id` 来选择某种（个）元素。

#### 3. 保存数据

将抓取到的数据存储到本地的 txt 文件中。

**Python 实现**

使用 `open` 方法读写[本地文件](title.txt)。

In [17]:
%%space request_demo_2

title = soup.title.text
with open("title.txt", "w") as f:
    f.write(title)

9

> **小拓展**：Python 中的文件读写  
> Python 中使用 `open` 方法开启一个读取/写入的流。可以指定读取写入编码，也可以读取写入字节数据。

## 数据整理

检查处理数据的重复值、缺失值和异常值等。

### 检测与处理重复值

- 记录重复：某几条记录的一个或多个特征值完全相同
- 特征重复：存在一个或者多个特征名称不同，但数据完全相同的情况

**Python 实现**

Python 的数据分析核心库 Pandas 提供了一个名为 `drop_duplicates` 的去重方法。

> [Pandas 官方文档](https://pandas.pydata.org/docs/reference/index.html)

```python
DataFrame.drop_duplicates(subset=None, *, keep='first', inplace=False, ignore_index=False) -> DataFrame | None
```

|参数名称|说明|
|-|-|
|`subset`|接受字符串或列表，表示进行去重的列，**默认为`None`**，表示全部列|
|`keep`|接受`'first'`, `'last'` 或 `False`，表示重复时保留哪个数据：`'first'` 保留第一个，<br/>`'last'` 保留最后一个，`'False'` 不保留数据。**默认为`'first'`**。|
|`inplace`|接受布尔值，表示是否在原表上进行操作，**默认为`False`**|
|`ignore_index`|接受布尔值，表示是否替换原表表头为数字编号，**默认为`False`**|

#### 实例
**目标**：对“某区共享单车数据”进行数据检测与去重处理

**1. 分析数据**

In [18]:
%%space bike_demo

import pandas

bike_data_frame = pandas.read_csv('csv/bike.csv')

print(bike_data_frame)

        index bike_id  datetime   date  month  season workingday local_name  \
0      598608  mr1706  00:00:00   6月1日      6  Summer        Yes         马桥   
1      614327   mr221  00:00:00   6月1日      6  Summer        Yes     闵行体育公园   
2      614328  mr5282  00:00:00   6月1日      6  Summer        Yes     闵行体育公园   
3      575272  mr5066  00:00:00   6月1日      6  Summer        Yes         颛桥   
4      578526  mr5674  00:00:00   6月1日      6  Summer        Yes     鑫都商业广场   
...       ...     ...       ...    ...    ...     ...        ...        ...   
83942  598599  mr4609  23:58:00  6月29日      6  Summer        Yes      闵行图书馆   
83943  598600   mr409  23:58:00  6月29日      6  Summer        Yes         颛桥   
83944  602447  mr2608  23:58:00  6月29日      6  Summer        Yes     麦多生活广场   
83945  602453  mr3521  23:59:00  6月29日      6  Summer        Yes     麦多生活广场   
83946  602454   mr825  23:59:00  6月29日      6  Summer        Yes     鑫都商业广场   

             weather isdaytime  temp_value temp_uni

以上面所示的数据为例，编号为“mr97068”的共享单车记录不仅出现了编号重复的情况，而且开锁时间等也有重复的情况出现。

**2. 确定方法**

因为同一辆单车在同一时间不可能被开锁两次，所以删除其中一条记录不会对数据造成不良影响，可以用 `drop_duplicates` 方法对数据进行去重处理。

**3. 编程与调试**

编写程序，对表中的数据进行去重处理，具体代码如下：

In [19]:
%%space bike_demo

bike_data_frame.drop_duplicates(subset=['bike_id','datetime'], keep='first', inplace=True)

print(bike_data_frame)

        index bike_id  datetime   date  month  season workingday local_name  \
0      598608  mr1706  00:00:00   6月1日      6  Summer        Yes         马桥   
1      614327   mr221  00:00:00   6月1日      6  Summer        Yes     闵行体育公园   
2      614328  mr5282  00:00:00   6月1日      6  Summer        Yes     闵行体育公园   
3      575272  mr5066  00:00:00   6月1日      6  Summer        Yes         颛桥   
4      578526  mr5674  00:00:00   6月1日      6  Summer        Yes     鑫都商业广场   
...       ...     ...       ...    ...    ...     ...        ...        ...   
83942  598599  mr4609  23:58:00  6月29日      6  Summer        Yes      闵行图书馆   
83943  598600   mr409  23:58:00  6月29日      6  Summer        Yes         颛桥   
83944  602447  mr2608  23:58:00  6月29日      6  Summer        Yes     麦多生活广场   
83945  602453  mr3521  23:59:00  6月29日      6  Summer        Yes     麦多生活广场   
83946  602454   mr825  23:59:00  6月29日      6  Summer        Yes     鑫都商业广场   

             weather isdaytime  temp_value temp_uni

### 检测与处理缺失值

缺失值是指数据中的某个或多个特征的值是不完整的。

删除法是常用的缺失值处理方法，它通过减少样本量来换取信息完整度，是一种较简单的缺失值处理方法。

**Python 实现**

Pandas 库提供了识别缺失值的方法 `isnull` 和识别非缺失值的方法 `notnull`。其具体语法如下：

```python
DataFrame.isnull() -> DataFrame
```
```python
DataFrame.notnull() -> DataFrame
```

返回一个全部由 `True` 和 `False` 构成的 `DataFrame`。`isnull` 方法会将缺失值标记为 `True`，将非缺失值标记为 `False`。`notnull` 方法则反之。

以前面分析的“某区共享单车数据”为例，编号为“mr96478”和“mr96254”的共享单车记录中出现了缺失日期(date)和站点名(local_name)这两个
特征值的情况。

In [20]:
%%space bike_demo

null_mapped_bike_data_frame = bike_data_frame.isnull()
not_null_mapped_bike_data_frame = bike_data_frame.notnull()

print(null_mapped_bike_data_frame)
print("=====================================================================================")
print(not_null_mapped_bike_data_frame)

       index  bike_id  datetime   date  month  season  workingday  local_name  \
0      False    False     False  False  False   False       False       False   
1      False    False     False  False  False   False       False       False   
2      False    False     False  False  False   False       False       False   
3      False    False     False  False  False   False       False       False   
4      False    False     False  False  False   False       False       False   
...      ...      ...       ...    ...    ...     ...         ...         ...   
83942  False    False     False  False  False   False       False       False   
83943  False    False     False  False  False   False       False       False   
83944  False    False     False  False  False   False       False       False   
83945  False    False     False  False  False   False       False       False   
83946  False    False     False  False  False   False       False       False   

       weather  isdaytime  

Pandas 库中提供了简便的删除缺失值的方法 `dropna`。通过参数控制，该方法既可以删除观测记录，也可以删除特征。其基本语法如下：

```python
DataFrame.dropna(*, axis=0, how='any', thresh=..., subset=None, inplace=False, ignore_index=False) -> DataFrame | None
```

|参数名称|说明|
|-|-|
|`axis`|接受0或1，表示轴向；0为删除观测记录（行）；1为删除特征（列）；**默认为0**|
|`how`|接收特定字符串，表示删除的形式；any表示只要有缺失值存在就执行删除操作，**默认为any**；all表示当且仅当全部为缺失值才执行删除操作|
|`thresh`|接收整型，表示最少需要的非空值数量，不能和how参数一起使用|
|`subset`|接收字符串或列表，表示要删除的列，**默认为`None`**|
|`inplace`|接收布尔值，表示是否在原表上进行操作，**默认为`False`**|
|`ignore_index`|接收布尔值，表示输出的列标签是否以下标的形式表示，**默认为`False`**|

#### 实例
**目标**：对“某区共享单车数据”进行数据检测与处理缺失值

**1. 分析数据**

以前面分析的“某区共享单车数据”为例，编号为“mr96478”和“mr96254”的共享单车记录中出现了缺失日期(date)和站点名(local_name)这两个
特征值的情况。

**2. 确定方法**

缺失日期和站点名两个特征值的记录对后续数据的统计的意义不大，可以用`dropna()`方法进行删除。

**3. 编程与调试**

编写程序，对test.csv中的数据处理缺失值，具体代码如下：

In [21]:
%%space bike_demo

bike_data_frame.dropna(axis=0,inplace=True)

print(bike_data_frame)

        index bike_id  datetime   date  month  season workingday local_name  \
0      598608  mr1706  00:00:00   6月1日      6  Summer        Yes         马桥   
1      614327   mr221  00:00:00   6月1日      6  Summer        Yes     闵行体育公园   
2      614328  mr5282  00:00:00   6月1日      6  Summer        Yes     闵行体育公园   
3      575272  mr5066  00:00:00   6月1日      6  Summer        Yes         颛桥   
4      578526  mr5674  00:00:00   6月1日      6  Summer        Yes     鑫都商业广场   
...       ...     ...       ...    ...    ...     ...        ...        ...   
83942  598599  mr4609  23:58:00  6月29日      6  Summer        Yes      闵行图书馆   
83943  598600   mr409  23:58:00  6月29日      6  Summer        Yes         颛桥   
83944  602447  mr2608  23:58:00  6月29日      6  Summer        Yes     麦多生活广场   
83945  602453  mr3521  23:59:00  6月29日      6  Summer        Yes     麦多生活广场   
83946  602454   mr825  23:59:00  6月29日      6  Summer        Yes     鑫都商业广场   

             weather isdaytime  temp_value temp_uni

### 检测与处理异常值

异常值是指数据中个别值的数值明显偏离其余的数值。检测异常值就是检验数据中是否有输入错误以及是否含有不合理的数据。一般使用箱形图或散点图能较清晰地观察到异常值的存在。

<img src="img/3/abnormal_values.png">

**异常值的处理方法**

- 直接将含有异常值的记录删除
- 用前后两个观测值的平均值修正该异常值
- 将异常值视为缺失值
- 利用处理缺失值的方法进行处理
- ......

## 数据读取与存储

不同的数据源需要使用不同的函数来读取。Pandas 内置了十余种数据源读取函数和对应的数据写入函数。

### 数据源

常见的数据源有文本文件（包括一般文本文件和CSV文件）、电子表格文件等。

- 文本文件
  - 由若干行字符构成的计算机文件
  - 顺序文件
  - CSV格式：一种用分隔符分隔的文件格式（因为其分隔符不一定是逗号，因此又被称为字符分隔文件格式）。CSV文件以纯文本形式存储表格数据（数值和文本）

### 文本文件的读取

Pandas库提供了`read_csv()`函数来读取CSV文件，其语法如下：

```python
pandas.read_csv(filepath, *, sep=',', header='infer', names=None, index_col=None, dtype=None, encoding='utf-8') -> DataFrame
```

|参数名称|说明|
|-|-|
|`filepath`|接收字符串，表示文件路径，无默认值|
|`sep`|接收字符串，表示分隔符，默认为`','`|
|`header`|接收整型，表示将某行数据作为列名，默认为`'infer'`，表示自动识别|
|`names`|接收数组，表示列名，默认为`None`|
|`index_col`|接收数组，表示输出时使用的列名，默认为`None`|
|`dtype`|接受字典，表示写入的数据类型，默认为`None`|
|`encoding`|接收特定字符串，表示存储文件的编码格式，默认为`utf-8`|

### 文本文件的存储

文本文件的存储与读取类似，对于结构化数据，可以通过Pandas库中的`to_csv()`函数实现以CSV文件格式进行存储。`to_csv()`函数的语法如下：

```python
DataFrame.to_csv(path_or_buf, sep=',', na_rep='', columns=None, header=True, index=True, index_label=None, mode='w', encoding=None) -> None

DataFrame.to_csv(path_or_buf=None, ...) -> str
```

|参数名称|说明|
|-|-|
|`path_or_buf`|接收字符串，表示文件路径，无默认值|
|`sep`|接收字符串，表示分隔符，默认为`','`|
|`na_rep`|接收字符串，表示缺失值，默认为空字符串|
|`columns`|接收列表，表示写出的列名，默认为`None`|
|`header`|接收布尔值，表示是否将列名列出，默认为`True`|
|`index`|接收布尔值，表示是否将行名（索引）列出，默认为`True`|
|`index_label`|接收序列，表示索引名，默认为`None`|
|`mode`|接收特定字符串，表示数据写入模式，默认为`'w'`|
|`encoding`|接收特定字符串，表示存储文件的编码格式，默认为`None`|

<br/>

> **小拓展**：关于 pandas 库中的数据类型
> Pandas 库是本章中使用的主要工具。导入 Pandas 库的语法为：
> ```python
> import pandas as pd
> ```
> - Pandas 库主要的数据类型
>   - Series
>     - 一维数据结构
>     - 用法与列表类似
>   - DataFrame
>     - 二维数据结构
>     - 典型结构为表格
> 
> 创建 DataFrame 的语法为：
> ```python
> df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=None)
> ```
> 例如，创建一个包含四种交通工具以及每种交通工具的日均客运量和占比的DataFrame，数据变量名称为df：
> ```python
> datas = [[969.2, 53.9], [602.9, 33.5], [208, 11.6], [13.6, 0.8]]
> indexs = ["轨道交通", "公共汽/电车", "出租车", "轮渡"]
> columns = ["日均客运量（万人次）", "占公共交通日均客运总量（%）"]
> ```
> 生成的 DataFrame 如下：
> ||日均客运量（万人次）|占公共交通日均客运总量（%）|
> |-|-|-|
> |轨道交通|969.2|53.9|
> |公共汽/电车|602.9|33.5|
> |出租车|208|11.6|
> |轮渡|13.6|0.8|
> 
> 这种方式会按用户输入数据的顺序来生成DataFrame，且具有行、列标题。如无特殊需求，我们通常会以这种方式生成 DataFrame。
> 
> 读取DataFrame一个列数据的语法为：
> ```python
> df[列标题]
> ```
> 要读取两个以上列数据的语法为：
> ```python
> df[[列标题1, 列标题2, ...]]
> ```


## 数据安全

1. 数据安全意识
2. 数据安全防护
    1. 数据加密
        - 利用加密算法和加密密钥将需要保护的数据（明文）转化成另外一种数据（密文）
        - 窃取者在没有密钥和不了解加密算法的情况下无法识别密文
    2. 数据脱敏
        - 是在**不影响数据分析结果准确性**的前提下，对需要保护的数据进行一定的变换操作，如替换、过滤或删除等，从而**降低数据的敏感性**，保护用户的隐私不被泄露
        <br/><br/><img src="img/3/desensitization.png">
    3. 访问控制
        - 确定用户身份及其所享有权限
        - 包含：
          - 身份验证
          - 授权
    4. 数据备份
        - 将整个系统的数据或者一部分关键数据通过一定的方法从主计算机系统的存储设备中复制到其他存储设备中
        - 往往需要定期定时进行，从而使得备份的数据能够保持最新的状态
    5. 异地容灾
        - 在相隔较远的地方，建立两套或多套功能相同的计算机系统，相互进行数据备份或应急时提供备用计算机服务